In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go




In [13]:
# Use the U.S. ticker FOR LIFE 360 INC (LIF)
# FIND DAILY PRICE AND PERCENT CHANGE
ticker = yf.Ticker("LIF")

info = ticker.info
current_price = info.get("currentPrice")
previous_close = info.get("previousClose")
percent_change = ((current_price - previous_close) / previous_close) * 100

print(f"Current Price: ${current_price:.2f}")
print(f"Change: {percent_change:.2f}%")



Current Price: $95.95
Change: -2.01%


In [14]:
# Extract key metrics
metrics = {
    "current_price" : current_price,
    "percent_change" : percent_change,
    "Market Cap": info.get("marketCap"),
    "P/E Ratio": info.get("trailingPE"),
    "EPS (TTM)": info.get("trailingEps"),
    "Revenue (TTM)": info.get("totalRevenue"),
    "Last Updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

df_metrics = pd.DataFrame(metrics, index=["Life360"])
display(df_metrics)

df_metrics.to_csv("../data/processed/lif_metrics.csv", index=False)

,current_price,percent_change,Market Cap,P/E Ratio,EPS (TTM),Revenue (TTM),Last Updated
Life360,95.95,-2.011846,7440792576,252.5,0.38,427399008,2025-10-30 16:34:25


In [15]:
#YTD PERFORMANCE DATA
# Get 1 year of daily data
hist = ticker.history(period="1y")

# 52-week stats
high_52 = hist["High"].max()
low_52 = hist["Low"].min()

# YTD performance
start_of_year = pd.Timestamp(datetime(datetime.now().year, 1, 1), tz='UTC')
ytd_data = hist.loc[hist.index >= pd.to_datetime(start_of_year)]
if not ytd_data.empty:
    start_price = ytd_data["Close"].iloc[0]
    current_price = ytd_data["Close"].iloc[-1]
    ytd_change = ((current_price - start_price) / start_price) * 100
else:
    ytd_change = None

df_ytd = pd.DataFrame({
    "Metric": ["52-Week High", "52-Week Low", "YTD Performance"],
    "Value": [
        f"${high_52:.2f}",
        f"${low_52:.2f}",
        f"{ytd_change:.2f}%" if ytd_change else "N/A"
    ]
})

display(df_ytd)



,Metric,Value
0,52-Week High,$112.54
1,52-Week Low,$29.62
2,YTD Performance,129.05%


In [16]:
# Stock Price Chart
# Do we include this here or straight in Tableau?

def get_stock_data(ticker_symbol, period):
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(period=period)
    hist.reset_index(inplace=True)
    hist['Date'] = pd.to_datetime(hist['Date'])
    return hist

def plot_stock_chart(hist, title="Life360 (LIF)"):
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=hist['Date'],
        open=hist['Open'],
        high=hist['High'],
        low=hist['Low'],
        close=hist['Close'],
        name='Price'
    ))

    fig.update_layout(
        title=title,
        yaxis_title="Price (USD)",
        xaxis_title="Date",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        hovermode="x unified",
        height=500
    )

    fig.show()



In [17]:
# Define periods
period_options = {
    "1D": "1d",
    "1W": "5d",
    "1M": "1mo",
    "6M": "6mo",
    "1Y": "1y",
    "5Y": "5y"
}

# Fetch all period data once
data_cache = {label: get_stock_data("LIF", period) for label, period in period_options.items()}

# Create initial figure (1Y default)
initial_period = "1Y"
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=data_cache[initial_period]['Date'],
    open=data_cache[initial_period]['Open'],
    high=data_cache[initial_period]['High'],
    low=data_cache[initial_period]['Low'],
    close=data_cache[initial_period]['Close'],
    name="LIF"
))

# Add dropdown buttons
fig.update_layout(
    title="Life360 (LIF) — Interactive Stock Chart",
    template="plotly_dark",
    yaxis_title="Price (USD)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=False,
    height=500,
    updatemenus=[
        dict(
            buttons=[
                dict(label=label,
                     method="update",
                     args=[
                         {"x": [data_cache[label]['Date']],
                          "open": [data_cache[label]['Open']],
                          "high": [data_cache[label]['High']],
                          "low": [data_cache[label]['Low']],
                          "close": [data_cache[label]['Close']]},
                         {"title": f"Life360 (LIFX) — {label} View"}
                     ])
                for label in period_options.keys()
            ],
            direction="down",
            showactive=True,
            x=0.15,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

fig.show()


In [18]:
fig.write_html("../data/processed/lifx_chart.html")

It's too complex... I'm going to make this more simpler and understandable.

In [19]:
def fetch_hist(symbol: str, period: str, interval: str):
    t = yf.Ticker(symbol)
    df = t.history(period=period, interval=interval)
    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"])
    return df

planning on creating kpi's for revenue, grossprofit, and net income
also want to show comparison of MAU's (monthly avg users)

In [24]:
ticker.financials

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,9.116000e+04,-1.680000e+05,0.000000e+00,-4.690298e+03,NaN
Tax Rate For Calcs,2.000000e-02,2.100000e-01,0.000000e+00,3.770000e-03,NaN
Normalized EBITDA,-2.425000e+06,-1.920000e+07,-8.829300e+07,-3.014200e+07,NaN
Total Unusual Items,4.558000e+06,-8.000000e+05,3.081000e+06,-1.244000e+06,NaN
Total Unusual Items Excluding Goodwill,4.558000e+06,-8.000000e+05,3.081000e+06,-1.244000e+06,NaN
Net Income From Continuing Operation Net Minority Interest,-4.555000e+06,-2.817100e+07,-9.162900e+07,-3.355700e+07,NaN
Reconciled Depreciation,1.010900e+07,9.983000e+06,9.199000e+06,8.760000e+05,NaN
Reconciled Cost Of Revenue,9.232700e+07,8.188100e+07,7.970700e+07,2.276800e+07,NaN
EBITDA,2.133000e+06,-2.000000e+07,-8.521200e+07,-3.138600e+07,NaN
EBIT,-7.976000e+06,-2.998300e+07,-9.441100e+07,-3.226200e+07,NaN


In [30]:
q_financials = ticker.quarterly_financials
q_financials.head()

,2025-06-30,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31
Tax Effect Of Unusual Items,266490.00,0.00,0.00,1131690.00,311640.00,NaN
Tax Rate For Calcs,0.21,0.21,0.21,0.21,0.21,NaN
Normalized EBITDA,3879000.00,5135000.00,8521000.00,-7859000.00,-1399000.00,NaN
Total Unusual Items,1269000.00,NaN,0.00,5389000.00,1484000.00,-2315000.0
Total Unusual Items Excluding Goodwill,1269000.00,NaN,0.00,5389000.00,1484000.00,-2315000.0


In [26]:
ticker.financials

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,9.116000e+04,-1.680000e+05,0.000000e+00,-4.690298e+03,NaN
Tax Rate For Calcs,2.000000e-02,2.100000e-01,0.000000e+00,3.770000e-03,NaN
Normalized EBITDA,-2.425000e+06,-1.920000e+07,-8.829300e+07,-3.014200e+07,NaN
Total Unusual Items,4.558000e+06,-8.000000e+05,3.081000e+06,-1.244000e+06,NaN
Total Unusual Items Excluding Goodwill,4.558000e+06,-8.000000e+05,3.081000e+06,-1.244000e+06,NaN
Net Income From Continuing Operation Net Minority Interest,-4.555000e+06,-2.817100e+07,-9.162900e+07,-3.355700e+07,NaN
Reconciled Depreciation,1.010900e+07,9.983000e+06,9.199000e+06,8.760000e+05,NaN
Reconciled Cost Of Revenue,9.232700e+07,8.188100e+07,7.970700e+07,2.276800e+07,NaN
EBITDA,2.133000e+06,-2.000000e+07,-8.521200e+07,-3.138600e+07,NaN
EBIT,-7.976000e+06,-2.998300e+07,-9.441100e+07,-3.226200e+07,NaN


I noticed a problem here. The adjusted ebitda isn't completely accurate. I looked at Life360's public reports and they all show to be positive so I'm thinking they calculate it differently. I'm going to visualize revenue instead so i don't give false information.

In [31]:
q_financials.loc["Total Revenue"]

2025-06-30    115381000.0
2025-03-31    103624000.0
2024-12-31    115529000.0
2024-09-30     92865000.0
2024-06-30     84863000.0
2024-03-31            NaN
Name: Total Revenue, dtype: float64

In [32]:
q_financials.tail()

,2025-06-30,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31
Other Gand A,17378000.0,15649000.0,16469000.0,15229000.0,14613000.0,NaN
Gross Profit,90501000.0,83549000.0,85506000.0,70012000.0,63626000.0,NaN
Cost Of Revenue,24880000.0,20075000.0,30023000.0,22853000.0,21237000.0,NaN
Total Revenue,115381000.0,103624000.0,115529000.0,92865000.0,84863000.0,NaN
Operating Revenue,100848000.0,90781000.0,102511000.0,83577000.0,77579000.0,NaN


new problem. something that really interests me as an investor is MAU and subscription based revenue. i believe this portion of their business is where they make their real cash. growth in this sector would indicate a profitable business model. regualar revenue can be temporary and trendy which is why i don't particularly care about it.

but the problem is yfinance doesn't have include this type of info. so now i have to implement tables from life360's public filings.
